# CDN Analysis Test for FEB 2021 DATA

# Introduction:

1.	What is the Cost per Passenger(CPP)?
2.	What is the total pax?
<br>    Answered
3.	What is the number of pax for each class?
<br>    Answered
4.	What is the total expenses?
<br>    Answered
5.	What is the total expenses per class?
<br>   Answered
6.	What is the total load factor?
<br>    Answered
7.	 What is the load factor per class? 
<br>    Answered
8.	Which station has the highest number of pax?
9.	Which station has the highest total price compared to unit price?
10.	Which station has the lowest number of pax compared to capacity?
11.	What is the number of pax in relation to time-of-flight date and hour?
12.	What is the impact of monthly changes?
13.	What are the top 10 highest volume items? 
14.	What are the top 10 highest expense items?
15.	What are the top 5 highest passenger count sectors?
16.	What is the highest order volume based on time?
17.	What is the correlation/impact of the load factor on cpp?
18.	What is the handling charge per route?
<br>    Answered
19.	How does the seasonally impact on cpp?
20.	What is the DOM cpp by menu cycle?
21.	SACC station ordered by volume and expenses?
22. what is the total handling charges?
<br>    Answered
23. what is the handling charges per Air Carft?
24. What are the monthly changes and their impact?

## Data Cleaning:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from time import sleep
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Initiate Data Frame for the current month and previous month
df = pd.read_excel("AVIF Feb 2021.xlsb", engine='pyxlsb')
df_p = pd.read_excel("AVIF Jan 2021.xlsb", engine='pyxlsb')

In [4]:
del_col = ['MAIN_SBD_CODE', 'AIRCRAFT_TYPE', 'TOP5', 'ROUTE_TYPE', 'CUST_NO', 'ROUTE', 'SR_FLT_NO', 'SR_SUFFIX', 'SR_FROM_CITY', 'SR_TO_CITY', 'SERVICE_NO', 'FPM_CODE', 'MENU_CODE', 'MENU_DESC', 'QTY', 'VAT_PRICE', 'MENU_CYCLE']
df.drop(del_col,axis=1, inplace=True)
df_p.drop(del_col,axis=1, inplace=True)

In [5]:
# print first 5 rows of data
df.head()

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE
0,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,CMY2024,DISP. GOWNS (SV),PC,43016.0,100,0.00,0.00,C5
1,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,#HAGCA,#HAGCA-HANDLING CHARGE,NaN,36423.0,NaN,101.10,101.10,C5
2,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,BBSKTB1545,BBSKTB1545-BREAD ASSORTED HLD 3,MFOIL,39137.0,1-4/1,3.59,7.18,C5
3,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,DSYI1505,DSYI1505-CAKE BEE STING (80GR),PO,39182.0,100,2.88,17.28,C5
4,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,ENYF1509,ENYF1509-HAMMOUR BAKED W/CARDINAL SAUCE,PO,39221.0,30,33.89,67.78,C5


In [6]:
df_p.head()

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE
0,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,CMY2024,DISP. GOWNS (SV),PC,43016,100,0.00,0.00,C5
1,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,#HAGCA,#HAGCA-HANDLING CHARGE,NaN,36423,NaN,101.10,101.10,C5
2,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,BBSKTB1544,BBSKTB1544-BREAD ASSORTED HLD 2,MFOIL,39143,1-4/1,5.21,10.42,C5
3,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,DSYI1504,DSYI1504-CAKE CHOCOLATE-BERRIES,PO,39186,100,4.78,28.68,C5
4,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,ENYF1515,ENYF1515-HAMMOUR FILLET KAPSA,PO,39205,30,33.32,66.64,C5


In [7]:
# drop international flights colunn
df_p.drop(df_p[df_p['TYPE_CODE'] == 'C1'].index, inplace=True)
df.drop(df[df['TYPE_CODE'] == 'C1'].index, inplace=True)

In [8]:
# drop the not related class values to the passenger count
df.drop(df[df.CLASS_CODE.isin(['CA', 'CO', 'CP', 'PX', 'PXC','SM'])].index, inplace=True)
df_p.drop(df_p[df_p.CLASS_CODE.isin(['CA', 'CO', 'CP', 'PX', 'PXC','SM'])].index, inplace=True)

In [9]:
# drop rows with item descreption equals to LAVATORY DISINFECTANT 750ML (SV) 
df_p.drop(df_p[df_p['ITEM_DESCRIPTION'] == 'LAVATORY DISINFECTANT 750ML (SV)'].index, inplace=True)
df.drop(df[df['ITEM_DESCRIPTION'] == 'LAVATORY DISINFECTANT 750ML (SV)'].index, inplace=True)

In [10]:
# Convert the date to datetime64
df_p['DBL_DATE'] = pd.to_datetime(df_p['DBL_DATE'] - 25569, unit = 'D')

In [11]:
df['DBL_DATE']  = pd.to_datetime(df['DBL_DATE'] - 25569, unit = 'D')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166469 entries, 70 to 694544
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            166469 non-null  int64         
 1   DBL_DATE          166469 non-null  datetime64[ns]
 2   FLT_NO            166469 non-null  object        
 3   ETD               166469 non-null  object        
 4   FROM_CITY         166469 non-null  object        
 5   TO_CITY           166469 non-null  object        
 6   LEGNO             166469 non-null  int64         
 7   CLASS_CODE        166469 non-null  object        
 8   CAPACITY          166469 non-null  int64         
 9   NO_PAX            166469 non-null  int64         
 10  ITEM_CODE         166469 non-null  object        
 11  ITEM_DESCRIPTION  166469 non-null  object        
 12  UOM               161637 non-null  object        
 13  SPECS_NO          166468 non-null  float64       
 14  ALO

In [13]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164387 entries, 62 to 828410
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            164387 non-null  int64         
 1   DBL_DATE          164387 non-null  datetime64[ns]
 2   FLT_NO            164387 non-null  object        
 3   ETD               164387 non-null  object        
 4   FROM_CITY         164387 non-null  object        
 5   TO_CITY           164387 non-null  object        
 6   LEGNO             164387 non-null  int64         
 7   CLASS_CODE        164387 non-null  object        
 8   CAPACITY          164387 non-null  int64         
 9   NO_PAX            164387 non-null  int64         
 10  ITEM_CODE         164387 non-null  object        
 11  ITEM_DESCRIPTION  164387 non-null  object        
 12  UOM               158951 non-null  object        
 13  SPECS_NO          164387 non-null  int64         
 14  ALO

In [14]:
# Convert specs_no to string 
df_p['SPECS_NO'] = df_p['SPECS_NO'].astype(str)

In [15]:
df['SPECS_NO'].fillna(0, inplace=True)
df['SPECS_NO'] = df['SPECS_NO'].astype('int32')

In [16]:
df['SPECS_NO'] = df['SPECS_NO'].astype(str)

In [17]:
# Convert price to integer 
df_p['PRICE'] = df_p['PRICE'].astype(int)
df['PRICE'] = df['PRICE'].astype(int)

In [18]:
# change numerical columns to int32 and float32 to reduce memory usage 
df['NO_PAX'] = df['NO_PAX'].astype('int32')
df_p['NO_PAX'] = df_p['NO_PAX'].astype('int32')

df['CAPACITY'] = df['CAPACITY'].astype('int32')
df_p['CAPACITY'] = df_p['CAPACITY'].astype('int32')

df['UNIT_PRICE'] = df['UNIT_PRICE'].astype('float32')
df_p['UNIT_PRICE'] = df_p['UNIT_PRICE'].astype('float32')

In [19]:
# Change CDN_NO to String
df['CDN_NO'] = df['CDN_NO'].astype(str)
df_p['CDN_NO'] = df_p['CDN_NO'].astype(str)

In [20]:
# Change LEGNO to String
df['LEGNO'] = df['LEGNO'].astype(str)
df_p['LEGNO'] = df_p['LEGNO'].astype(str)

In [21]:
#Check memory usage for df
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166469 entries, 70 to 694544
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            166469 non-null  object        
 1   DBL_DATE          166469 non-null  datetime64[ns]
 2   FLT_NO            166469 non-null  object        
 3   ETD               166469 non-null  object        
 4   FROM_CITY         166469 non-null  object        
 5   TO_CITY           166469 non-null  object        
 6   LEGNO             166469 non-null  object        
 7   CLASS_CODE        166469 non-null  object        
 8   CAPACITY          166469 non-null  int32         
 9   NO_PAX            166469 non-null  int32         
 10  ITEM_CODE         166469 non-null  object        
 11  ITEM_DESCRIPTION  166469 non-null  object        
 12  UOM               161637 non-null  object        
 13  SPECS_NO          166469 non-null  object        
 14  ALO

In [22]:
#Check memory usage for df_p
df_p.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164387 entries, 62 to 828410
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            164387 non-null  object        
 1   DBL_DATE          164387 non-null  datetime64[ns]
 2   FLT_NO            164387 non-null  object        
 3   ETD               164387 non-null  object        
 4   FROM_CITY         164387 non-null  object        
 5   TO_CITY           164387 non-null  object        
 6   LEGNO             164387 non-null  object        
 7   CLASS_CODE        164387 non-null  object        
 8   CAPACITY          164387 non-null  int32         
 9   NO_PAX            164387 non-null  int32         
 10  ITEM_CODE         164387 non-null  object        
 11  ITEM_DESCRIPTION  164387 non-null  object        
 12  UOM               158951 non-null  object        
 13  SPECS_NO          164387 non-null  object        
 14  ALO

### Import and Clean Summer Schedule: 

This step will help in categorizing flights as per meal policy

In [23]:
# import the summer schedule to get block time
summerSchedule = pd.read_excel('summer_schedule.xlsx')

In [24]:
# Print first five rows
summerSchedule.head()

,Flt Desg,Eff Date,Dis Date,Freq,Dept Arp,Dept Time,Arvl Arp,Arrv Time,Subfleet,Block Time
0,SV 0020,2021-06-03,2021-10-28,1..4...,JFK,18:00:00,JED,05:40:00,SV 77Z,11:40:00
1,SV 0021,2021-06-03,2021-10-28,1..4...,JED,02:55:00,JFK,15:50:00,SV 77Z,12:55:00
2,SV 0022,2021-06-01,2021-10-29,.2..5..,JFK,18:00:00,RUH,05:55:00,SV 77Z,11:55:00
3,SV 0023,2021-06-01,2021-06-29,.2..5..,RUH,02:25:00,JFK,15:50:00,SV 77Z,13:25:00
4,SV 0023,2021-07-02,2021-10-29,.2..5..,RUH,02:25:00,JFK,15:50:00,SV 77Z,13:25:00


In [25]:
# rename sector columns to be aligned with the other dataframe to be joined with 
summerSchedule.rename(columns={"Dept Arp": "FROM_CITY", "Arvl Arp": "TO_CITY"}, inplace=True)

In [26]:
# drop unused columns
summerSchedule = summerSchedule.drop(columns=['Eff Date', 'Dis Date','Freq','Dept Time','Arrv Time','Subfleet', 'Flt Desg'])

In [27]:
# Print first five rows
summerSchedule.head()

,FROM_CITY,TO_CITY,Block Time
0,JFK,JED,11:40:00
1,JED,JFK,12:55:00
2,JFK,RUH,11:55:00
3,RUH,JFK,13:25:00
4,RUH,JFK,13:25:00


In [28]:
# remove duplicates to ensure merge will become 1:m
summerSchedule.drop_duplicates(subset=['FROM_CITY', 'TO_CITY'], inplace=True)

In [29]:
# print new df informations
summerSchedule.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 3266
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FROM_CITY   318 non-null    object
 1   TO_CITY     318 non-null    object
 2   Block Time  318 non-null    object
dtypes: object(3)
memory usage: 9.9+ KB


In [30]:
#perform the join of the dataframes
df = df.merge(summerSchedule, on=['FROM_CITY','TO_CITY'],how = 'inner')
df_p = df_p.merge(summerSchedule, on=['FROM_CITY','TO_CITY'],how = 'inner')

In [31]:
# Convert block time into minutes
df_p['Block Time'] =  pd.to_datetime(df_p['Block Time'].astype(str))
df['Block Time'] =  pd.to_datetime(df['Block Time'].astype(str))

In [32]:
df_p['Block Time'] = df_p['Block Time'].dt.hour*60+df_p['Block Time'].dt.minute
df['Block Time'] = df['Block Time'].dt.hour*60+df['Block Time'].dt.minute

## Create a paxFarme:

In [33]:
#Create a copy of the main data farme and store it in the variable df_copy
df_copy = df.copy()
df_p_copy = df_p.copy()

In [34]:
# make an instance of df_copy and call it paxFrame
paxFrame = df_copy[['CDN_NO', 'DBL_DATE', 'FLT_NO', 'TYPE_CODE', 'FROM_CITY', 'TO_CITY', 'CAPACITY', 'NO_PAX', 'CLASS_CODE', 'Block Time']]
paxFrameP = df_p_copy[['CDN_NO', 'DBL_DATE', 'FLT_NO', 'TYPE_CODE', 'FROM_CITY', 'TO_CITY', 'CAPACITY', 'NO_PAX', 'CLASS_CODE', 'Block Time']]

In [35]:
#check the information of paxFrame
paxFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166073 entries, 0 to 166072
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   CDN_NO      166073 non-null  object        
 1   DBL_DATE    166073 non-null  datetime64[ns]
 2   FLT_NO      166073 non-null  object        
 3   TYPE_CODE   166073 non-null  object        
 4   FROM_CITY   166073 non-null  object        
 5   TO_CITY     166073 non-null  object        
 6   CAPACITY    166073 non-null  int32         
 7   NO_PAX      166073 non-null  int32         
 8   CLASS_CODE  166073 non-null  object        
 9   Block Time  166073 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 12.7+ MB


In [36]:
#check the information of paxFrameP
paxFrameP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163848 entries, 0 to 163847
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   CDN_NO      163848 non-null  object        
 1   DBL_DATE    163848 non-null  datetime64[ns]
 2   FLT_NO      163848 non-null  object        
 3   TYPE_CODE   163848 non-null  object        
 4   FROM_CITY   163848 non-null  object        
 5   TO_CITY     163848 non-null  object        
 6   CAPACITY    163848 non-null  int32         
 7   NO_PAX      163848 non-null  int32         
 8   CLASS_CODE  163848 non-null  object        
 9   Block Time  163848 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 12.5+ MB


In [37]:
# Check the number of passengers
paxFrame['NO_PAX'].sum()

5278920

In [38]:
# Check the number of passengers
paxFrameP['NO_PAX'].sum()

5267624

In [39]:
# the number of passenger shown above is un realistic becuase of duplicated 'NO_PAX' which is related to menu items
# Therefore this code is used to remove duplicated rows
paxFrame.drop_duplicates(subset=['FROM_CITY', 'TO_CITY','DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'], inplace=True)

<ipython-input-39-f6218a3185f0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paxFrame.drop_duplicates(subset=['FROM_CITY', 'TO_CITY','DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'], inplace=True)


In [40]:
# the number of passenger shown above is un realistic becuase of duplicated 'NO_PAX' which is related to menu items
# Therefore this code is used to remove duplicated rows
paxFrameP.drop_duplicates(subset=['FROM_CITY', 'TO_CITY', 'DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'], inplace=True)

<ipython-input-40-3835e93ad6d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paxFrameP.drop_duplicates(subset=['FROM_CITY', 'TO_CITY', 'DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'], inplace=True)


In [41]:
# Check the paxFrame after removing duplicates
paxFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14105 entries, 0 to 166072
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CDN_NO      14105 non-null  object        
 1   DBL_DATE    14105 non-null  datetime64[ns]
 2   FLT_NO      14105 non-null  object        
 3   TYPE_CODE   14105 non-null  object        
 4   FROM_CITY   14105 non-null  object        
 5   TO_CITY     14105 non-null  object        
 6   CAPACITY    14105 non-null  int32         
 7   NO_PAX      14105 non-null  int32         
 8   CLASS_CODE  14105 non-null  object        
 9   Block Time  14105 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 1.1+ MB


In [42]:
# Check the paxFrame after removing duplicates
paxFrameP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15423 entries, 0 to 163847
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CDN_NO      15423 non-null  object        
 1   DBL_DATE    15423 non-null  datetime64[ns]
 2   FLT_NO      15423 non-null  object        
 3   TYPE_CODE   15423 non-null  object        
 4   FROM_CITY   15423 non-null  object        
 5   TO_CITY     15423 non-null  object        
 6   CAPACITY    15423 non-null  int32         
 7   NO_PAX      15423 non-null  int32         
 8   CLASS_CODE  15423 non-null  object        
 9   Block Time  15423 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 1.2+ MB


## Number of Passengers:

#### Total Number of Passengers:

In [43]:
paxCountP = paxFrameP['NO_PAX'].sum()
print('Total number of passenger for the previous month {:,}'.format(paxCountP))

Total number of passenger for the previous month 909,771


In [44]:
paxCount = paxFrame['NO_PAX'].sum()
print('Total number of passenger for the current month {:,}'.format(paxCount))

Total number of passenger for the current month 699,428


##### Number of Passengers per Class for Previous Month:

In [45]:
fc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'FC'].sum()
fc_weight_p = fc_count_p/paxCountP
print('First class passengers count equals {:,} PAX, and its weight is {}'.format(fc_count_p, np.round(fc_weight_p, 3)))

bc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'JC'].sum()
bc_weight_p = bc_count_p/paxCountP
print('Business class passengers count equals {:,} PAX, and its weight is {}'.format(bc_count_p, np.round(bc_weight_p, 2)))

yc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'YC'].sum()
yc_weight_p = yc_count_p/paxCountP
print('Economy class passengers count equals {:,} PAX, and its weight is {}'.format(yc_count_p, np.round(yc_weight_p, 2)))

First class passengers count equals 31 PAX, and its weight is 0.0
Business class passengers count equals 100,696 PAX, and its weight is 0.11
Economy class passengers count equals 809,044 PAX, and its weight is 0.89


##### Number of Passengers per Class for Current Month:

In [46]:
fc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'FC'].sum()
fc_weight = fc_count/paxCount
print('First class passengers count equals {:,} PAX, and its weight is {}'.format(fc_count, np.round(fc_weight, 3)))

bc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'JC'].sum()
bc_weight = bc_count/paxCount
print('Business class passengers count equals {:,} PAX, and its weight is {}'.format(bc_count, np.round(bc_weight, 2)))

yc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'YC'].sum()
yc_weight = yc_count/paxCount
print('Economy class passengers count equals {:,} PAX, and its weight is {}'.format(yc_count, np.round(yc_weight, 2)))

First class passengers count equals 5 PAX, and its weight is 0.0
Business class passengers count equals 73,772 PAX, and its weight is 0.11
Economy class passengers count equals 625,651 PAX, and its weight is 0.89


### Seat Capacity:

#### Total Capacity:

In [47]:
seatCapacityP = paxFrameP['CAPACITY'].sum()
print('Total capacity for domestic flights for previous month equals {:,} seat.'.format(seatCapacityP))

Total capacity for domestic flights for previous month equals 1,452,349 seat.


In [48]:
seatCapacity = paxFrame['CAPACITY'].sum()
print('Total capacity for domestic flights for current month equals {:,} seat.'.format(seatCapacity))

Total capacity for domestic flights for current month equals 1,299,321 seat.


##### Capacity per Class Previous Month:

In [49]:
fc_capacity_p = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'FC'].sum()
print('First class seat capacity equals {:,} seat.'.format(fc_capacity_p))

bc_capacity_p  = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'JC'].sum()
print('Business class seat capacity equals {:,} seat.'.format(bc_capacity_p))

yc_capacity_p  = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'YC'].sum()
print('Economy class seat capacity equals {:,} seat.'.format(yc_capacity_p))

First class seat capacity equals 60 seat.
Business class seat capacity equals 129,782 seat.
Economy class seat capacity equals 1,322,507 seat.


##### Capacity per Class Current month:

In [50]:
fc_capacity = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'FC'].sum()
print('First class seat capacity equals {:,} seat.'.format(fc_capacity))

bc_capacity  = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'JC'].sum()
print('Business class seat capacity equals {:,} seat.'.format(bc_capacity))

yc_capacity  = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'YC'].sum()
print('Economy class seat capacity equals {:,} seat.'.format(yc_capacity))

First class seat capacity equals 24 seat.
Business class seat capacity equals 122,162 seat.
Economy class seat capacity equals 1,177,135 seat.


### Load Factor:

#### Total Load Factor:

In [51]:
loadFactor = (paxCountP/seatCapacityP)*100
print('domestic flights load factor for previous month {}%'.format(np.round(loadFactor,2)))

domestic flights load factor for previous month 62.64%


In [52]:
loadFactor = (paxCount/seatCapacity)*100
print('domestic flights load factor for current month {}%'.format(np.round(loadFactor,2)))

domestic flights load factor for current month 53.83%


##### Load Factor per Class Previous Month:

In [53]:
fc_lf_p = (fc_count_p/fc_capacity_p)*100
print('First class load factor equals {}%'.format(np.round(fc_lf_p, 2)))

bc_lf_p = (bc_count_p/bc_capacity_p)*100
print('Business class load factor equals {}%'.format(np.round(bc_lf_p, 2)))

yc_lf_p  = (yc_count_p/yc_capacity_p)*100
print('Economy class load factor equals {}%'.format(np.round(yc_lf_p, 2)))

First class load factor equals 51.67%
Business class load factor equals 77.59%
Economy class load factor equals 61.18%


##### Load Factor per Class Current Month:

In [54]:
fc_lf = (fc_count/fc_capacity)*100
print('First class load factor equals {}%'.format(np.round(fc_lf, 2)))

bc_lf = (bc_count/bc_capacity)*100
print('Business class load factor equals {}%'.format(np.round(bc_lf, 2)))

yc_lf  = (yc_count/yc_capacity)*100
print('Economy class load factor equals {}%'.format(np.round(yc_lf, 2)))

First class load factor equals 20.83%
Business class load factor equals 60.39%
Economy class load factor equals 53.15%


# Expenses:

### Total Expenses:

In [55]:
domExpP = df_p['PRICE'][df_p['TYPE_CODE'] == 'C5'].sum()
print('The toal expenses for domestic flights for previous month equals SAR {:,}'.format(domExpP))

The toal expenses for domestic flights for previous month equals SAR 26,300,126


In [56]:
domExp = df['PRICE'][df['TYPE_CODE'] == 'C5'].sum()
print('The toal expenses for domestic flights for current month equals SAR {:,}'.format(domExp))

The toal expenses for domestic flights for current month equals SAR 22,006,336


##### Handling Charges:

In [57]:
handlingChargesP = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5')&(df_p.ITEM_CODE.isin(['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI']))].sum()
handlingPerPaxP = handlingChargesP/paxCountP
print('The total handling charges for previous month equals to {:,} and handling per pax equals to {}'.format(handlingChargesP, np.round(handlingPerPaxP, 2)))

The total handling charges for previous month equals to 15,291,422 and handling per pax equals to 16.81


In [58]:
handlingCharges = df['PRICE'][(df['TYPE_CODE'] == 'C5')&(df.ITEM_CODE.isin(['#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI']))].sum()
handlingPerPax = handlingCharges/paxCount
print('The total handling charges for current month equals to {:,} and handling per pax equals to {}'.format(handlingCharges, np.round(handlingPerPax, 2)))

The total handling charges for current month equals to 13,460,149 and handling per pax equals to 19.24


##### Expenses per Class for Previous Month:

In [59]:
fc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'FC')].sum()
print('First class expenses equals to SAR {:,}'.format(fc_exp_p))

bc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'JC')].sum()
print('Bsiness class expenses equals to SAR {:,}'.format(bc_exp_p))

yc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'YC')].sum() - handlingChargesP
print('Economy class expenses equals to SAR {:,}'.format(yc_exp_p))

First class expenses equals to SAR 2,062
Bsiness class expenses equals to SAR 5,100,282
Economy class expenses equals to SAR 5,906,360


##### Expenses per Class for Current month:

In [60]:
fc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'FC')].sum()
print('First class expenses equals to SAR {:,}'.format(fc_exp))

bc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'JC')].sum()
print('Bsiness class expenses equals to SAR {:,}'.format(bc_exp))

yc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'YC')].sum() - handlingCharges
print('Economy class expenses equals to SAR {:,}'.format(yc_exp))

First class expenses equals to SAR 750
Bsiness class expenses equals to SAR 3,875,820
Economy class expenses equals to SAR 4,669,617


##### Cost per Passenger (CPP) for Previous Month:

In [61]:
fc_cpp_p = fc_exp_p / fc_count_p
fc_cpp_h_p = fc_cpp_p + handlingPerPaxP
print('First class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(fc_cpp_p, 2), np.round(fc_cpp_h_p, 2)))

bc_cpp_p = bc_exp_p / bc_count_p
bc_cpp_h_p = bc_cpp_p + handlingPerPaxP
print('Business class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(bc_cpp_p, 2), np.round(bc_cpp_h_p, 2)))

yc_cpp_p = yc_exp_p / yc_count_p
yc_cpp_h_p = yc_cpp_p + handlingPerPaxP
print('Economy class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(yc_cpp_p, 2), np.round(yc_cpp_h_p, 2)))

avgCppP = domExpP/paxCountP
avgCppNoHacP = avgCppP - handlingPerPaxP
print('The average CPP equals SAR {} and average meal CPP equals SAR {} .'.format(np.round(avgCppP,2), np.round(avgCppNoHacP, 2)))

First class meal cpp equals to SAR 66.52 and SAR 83.32 including handling charges.
Business class meal cpp equals to SAR 50.65 and SAR 67.46 including handling charges.
Economy class meal cpp equals to SAR 7.3 and SAR 24.11 including handling charges.
The average CPP equals SAR 28.91 and average meal CPP equals SAR 12.1 .


##### Cost per Passenger (CPP) for Current Month:

In [62]:
fc_cpp = fc_exp / fc_count
fc_cpp_h = fc_cpp + handlingPerPax
print('First class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(fc_cpp, 2), np.round(fc_cpp_h, 2)))

bc_cpp = bc_exp / bc_count
bc_cpp_h = bc_cpp + handlingPerPax
print('Business class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(bc_cpp, 2), np.round(bc_cpp_h, 2)))

yc_cpp = yc_exp / yc_count
yc_cpp_h = yc_cpp + handlingPerPax
print('Economy class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(yc_cpp, 2), np.round(yc_cpp_h, 2)))

avgCpp = domExp/paxCount
avgCppNoHac = avgCpp - handlingPerPax
print('The average CPP equals SAR {} and average meal CPP equals SAR {} .'.format(np.round(avgCpp,2), np.round(avgCppNoHac, 2)))

First class meal cpp equals to SAR 150.0 and SAR 169.24 including handling charges.
Business class meal cpp equals to SAR 52.54 and SAR 71.78 including handling charges.
Economy class meal cpp equals to SAR 7.46 and SAR 26.71 including handling charges.
The average CPP equals SAR 31.46 and average meal CPP equals SAR 12.22 .


### What are the monthly changes and their impact?

### Create Menu Frames by Service:

In [126]:
# make a copy of the invoice df and df_p
df_three = df.copy()
df_p_three = df_p.copy()

In [127]:
# Define variables for filteration  
handling_item_codes =['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI', '#HAEE']
# Flights that supposed to be beverage but follows meal policy
meal_exception = ['RUH-MED-RUH', 'RUH-YNB-RUH', 'RUH-ABT-RUH', 'RUH-BHH-RUH', 'RUH-TIF-RUH', 'MED-RUH-MED' ]
# Flights that supposed to be meal but follows beverage policy
beverage_exception = ['JED-ELQ-JED', 'JED-HAS-JED', 'JED-EAM-JED', 'JED-WAE-JED', 'JED-GIZ-JED', 'RUH-RAH-RUH' ]
# Add route column for comparsion purposes 
df_three['ROUTE'] = df_three['FROM_CITY'] + '-' + df_three['TO_CITY'] + '-' + df_three['FROM_CITY'] 
df_p_three['ROUTE'] = df_p_three['FROM_CITY'] + '-' + df_p_three['TO_CITY'] + '-' + df_p_three['FROM_CITY'] 

In [141]:
# create beverages data frame for YC
bev_frame_yc = df_three[((df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(meal_exception))) | ((df_three['Block Time'] <= 89) & (df_three.ROUTE.isin(beverage_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
bev_frame_p_yc = df_p_three[((df_p_three['Block Time'] <= 89) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(meal_exception))) | ((df_p_three['Block Time'] <= 89) & (df_p_three.ROUTE.isin(beverage_exception)) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

# Remove duplicates rows
bev_frame_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)
bev_frame_p_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)

# drop unneeded columns
bev_frame_yc = bev_frame_yc.drop(columns=['CDN_NO', 'ETD', 'FLT_NO' ,'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
bev_frame_p_yc = bev_frame_p_yc.drop(columns=['CDN_NO', 'ETD', 'FLT_NO','FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

#Keep leg number 1 only 
bev_frame_yc = bev_frame_yc.drop(bev_frame_yc[bev_frame_yc.LEGNO.isin(['2', '3', '4'])].index)
bev_frame_p_yc = bev_frame_p_yc.drop(bev_frame_p_yc[bev_frame_p_yc.LEGNO.isin(['2', '3', '4'])].index)

#Change index values to SPEC_NO
bev_frame_yc.set_index(bev_frame_yc['SPECS_NO'], inplace = True)
bev_frame_p_yc.set_index(bev_frame_p_yc['SPECS_NO'], inplace = True)

# compare this month's items by last month's items, using the isin() function

for i in range(1,32):
    daily_menu = bev_frame_yc[bev_frame_yc.DBL_DATE.dt.day == i]
    daily_p_menu = bev_frame_p_yc[bev_frame_p_yc.DBL_DATE.dt.day == i]
    changes_list_bev_yc = pd.concat([changes_list_bev_yc, (daliy_menu[~(daliy_menu.isin(daliy_p_menu))] ), (daliy_p_menu[~(daliy_p_menu.isin(daliy_menu))])])
# remove empty rows using the dropna() function
changes_list_bev_yc.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)
changes_list_bev_yc.drop_duplicates(subset= 'SPECS_NO', inplace =True)
display(changes_list_bev_yc)

<ipython-input-141-e6e78dd38461>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bev_frame_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)
<ipython-input-141-e6e78dd38461>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bev_frame_p_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)


,DBL_DATE,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE
SPECS_NO,,,,,,,,
36622,2021-01-30,1,DP06A,DP06A-ICE CUBES 5KG/BAG,BAG,36622,1-60/1,10.35
35277,2021-01-30,1,SD02,SD02-PEPSI COLA,CAN,35277,1-60/2,3.17
35279,2021-01-30,1,SD03,SD03-7UP (355ML),CAN,35279,1-60/2,3.18
35278,2021-01-30,1,SD11,SD11-DIET PEPSI,CAN,35278,1-60/1,3.18
35280,2021-01-30,1,SD12,SD12-DIET 7UP,CAN,35280,1-60/1,3.18
36757,2021-01-30,1,SD17R,SD17R-JUICE APPLE CEASAR 1LTR,LTR,36757,1-60/2,5.04
41288,2021-01-30,1,SD1802,WATER MINERAL HAYAT (1.5LTR),BTL,41288,1-60/7,1.13
41756,2021-01-30,1,SD18R,SD18R-MANGO CEASER JUICE 1 LTR,PKT,41756,1-60/2,9.66
41757,2021-01-30,1,SD19R,SD19R-ORANGE JUICE CEASAR(1 LTR),LTR,41757,1-60/3,5.04


### Beverage Flights:

#### YC beverage flight impact:

In [143]:
# create beverages data frame for YC
bev_frame_yc = df_three[((df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(meal_exception))) | ((df_three['Block Time'] <= 89) & (df_three.ROUTE.isin(beverage_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
bev_frame_p_yc = df_p_three[((df_p_three['Block Time'] <= 89) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(meal_exception))) | ((df_p_three['Block Time'] <= 89) & (df_p_three.ROUTE.isin(beverage_exception)) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

# Remove duplicates rows
bev_frame_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)
bev_frame_p_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)

# drop unneeded columns
bev_frame_yc = bev_frame_yc.drop(columns=['CDN_NO', 'ETD', 'FLT_NO' ,'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
bev_frame_p_yc = bev_frame_p_yc.drop(columns=['CDN_NO', 'ETD', 'FLT_NO','FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

#Keep leg number 1 only 
bev_frame_yc = bev_frame_yc.drop(bev_frame_yc[bev_frame_yc.LEGNO.isin(['2', '3', '4'])].index)
bev_frame_p_yc = bev_frame_p_yc.drop(bev_frame_p_yc[bev_frame_p_yc.LEGNO.isin(['2', '3', '4'])].index)

#Change index values to SPEC_NO
bev_frame_yc.set_index(bev_frame_yc['SPECS_NO'], inplace = True)
bev_frame_p_yc.set_index(bev_frame_p_yc['SPECS_NO'], inplace = True)

# compare this month's items by last month's items, using the isin() function
for i in range(1,32):
    daily_menu = bev_frame_yc[bev_frame_yc.DBL_DATE.dt.day == i]
    daily_p_menu = bev_frame_p_yc[bev_frame_p_yc.DBL_DATE.dt.day == i]
    changes_list_bev_yc = pd.concat([changes_list_bev_yc, (daliy_menu[~(daliy_menu.isin(daliy_p_menu))] ), (daliy_p_menu[~(daliy_p_menu.isin(daliy_menu))])])


# remove empty rows using the dropna() function
changes_list_bev_yc.dropna(subset=['SPECS_NO'], inplace=True)

#drop duplicated rows
changes_list_bev_yc.drop_duplicates(subset=['SPECS_NO'], inplace = True)

#Create another frame for the new month to calculate the impact
bev_frame_yc_im = df_three[((df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(meal_exception))) | ((df_three['Block Time'] <= 89) & (df_three.ROUTE.isin(beverage_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
#Create another frame for the previous month to calculate the impact
bev_frame_p_yc_im = df_p_three[((df_p_three['Block Time'] <= 89) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(meal_exception))) | ((df_p_three['Block Time'] <= 89) & (df_p_three.ROUTE.isin(beverage_exception)) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

#Get the total cost of each new item
changes_list_bev_yc['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list_bev_yc['CPP_IMPACT'] = 0 # Add impact column to change_list
changes_list_bev_yc['STATUS'] = ''
bev_yc_cost_im = 0  # place holder for total cost impact for each item

for x,z in enumerate(changes_list_bev_yc['SPECS_NO'].values): # loop to get the specs number for new items
    if z in bev_frame_yc_im['SPECS_NO'].values:
        for i, y in enumerate(bev_frame_yc_im['PRICE'][bev_frame_yc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            bev_yc_cost_im = bev_yc_cost_im + y
            changes_list_bev_yc['STATUS'].iloc[x] = 'Add'
    else:
        for i, y in enumerate(bev_frame_p_yc_im['PRICE'][bev_frame_p_yc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            bev_yc_cost_im = bev_yc_cost_im - y
            changes_list_bev_yc['STATUS'].iloc[x] = 'Remove'
    # claculate the CPP for the current spec number    
    cpp_im = bev_yc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list_bev_yc['TOTAL_COST'].iloc[x] = bev_yc_cost_im
    # Add the CPP impact to the current spec number
    changes_list_bev_yc['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    bev_yc_cost_im = 0

# Remove duplicates rows
bev_frame_yc.drop_duplicates(subset=['SPECS_NO'], inplace= True)
bev_frame_p_yc.drop_duplicates(subset=['SPECS_NO'], inplace= True)

#Month variable
current_month = bev_frame_yc.DBL_DATE.dt.month_name().iloc[0]
previous_month = bev_frame_p_yc.DBL_DATE.dt.month_name().iloc[0]

# drop unneeded column
bev_frame_yc = bev_frame_yc.drop(columns=['DBL_DATE'])
bev_frame_p_yc = bev_frame_p_yc.drop(columns=['DBL_DATE'])
changes_list_bev_yc = changes_list_bev_yc.drop(columns=['DBL_DATE', 'UNIT_PRICE'])

#Reset index values 
bev_frame_yc.reset_index(drop= True, inplace = True)
bev_frame_p_yc.reset_index(drop= True, inplace = True)
changes_list_bev_yc.reset_index(drop= True, inplace = True)


sleep(1) # Time in seconds

if changes_list_bev_yc['ITEM_DESCRIPTION'].count() > 0:
    # print previous month dataframe
    print('{} menu:'.format(previous_month))
    display(bev_frame_p_yc)
    # print current month dataframe
    print('{} menu:'.format(current_month))
    display(bev_frame_yc)
    print('The following is the changes and impact breakdown:')
    display(changes_list_bev_yc)
    print('Total cost impact on beverage flights economy equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list_bev_yc['TOTAL_COST'].sum(), changes_list_bev_yc['CPP_IMPACT'].sum()))

else:
    # print previous month dataframe
    print('Previous month menu:')
    display(bev_frame_p_yc)
    # print current month dataframe
    print('Current month menu:')
    display(bev_frame_yc)
    print('No changes observed')

<ipython-input-143-26f26ad9ce4f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bev_frame_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)
<ipython-input-143-26f26ad9ce4f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bev_frame_p_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)
C:\Users\kh-mo\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  se

January menu:


,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE
0,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41
1,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38
2,1,SD1802,WATER MINERAL HAYAT (1.5LTR),BTL,41288,NaN,1.13
3,1,CMY2025,SANITIZER BOTTLE 400ML (SV),BTL,43017,NaN,0.00
4,1,#SDICE1,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52
5,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,NaN,1.38
6,1,DP06A,DP06A-ICE CUBES 5KG/BAG,BAG,36622,1-60/1,10.35
7,1,SD02,SD02-PEPSI COLA,CAN,35277,1-60/2,3.17
8,1,SD03,SD03-7UP (355ML),CAN,35279,1-60/2,3.18
9,1,SD11,SD11-DIET PEPSI,CAN,35278,1-60/1,3.18


February menu:


,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE
0,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41
1,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38
2,1,#SDICE1,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52
3,1,CMY2021,DISPOSABLE COVID-19 BOX SMALL (SV),PC,42893,100,0.00
4,1,SD1801,"TSC, SD1801-WATER CUP W/O LOGO HAYAT (125ML)",CUP,41289,100,0.41
5,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.38
6,1,SNYIC2041,S/W RECT TURKEY SMOKED-PEPPERS/THREE CHEESE-,PO,42855,100,10.97


The following is the changes and impact breakdown:


,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,TOTAL_COST,CPP_IMPACT,STATUS
0,1,DP06A,DP06A-ICE CUBES 5KG/BAG,BAG,36622,1-60/1,-20,-0.000029,Remove
1,1,SD02,SD02-PEPSI COLA,CAN,35277,1-60/2,-12,-0.000017,Remove
2,1,SD03,SD03-7UP (355ML),CAN,35279,1-60/2,-12,-0.000017,Remove
3,1,SD11,SD11-DIET PEPSI,CAN,35278,1-60/1,-6,-0.000009,Remove
4,1,SD12,SD12-DIET 7UP,CAN,35280,1-60/1,-6,-0.000009,Remove
5,1,SD17R,SD17R-JUICE APPLE CEASAR 1LTR,LTR,36757,1-60/2,-20,-0.000029,Remove
6,1,SD1802,WATER MINERAL HAYAT (1.5LTR),BTL,41288,1-60/7,-55,-0.000079,Remove
7,1,SD18R,SD18R-MANGO CEASER JUICE 1 LTR,PKT,41756,1-60/2,-38,-0.000054,Remove
8,1,SD19R,SD19R-ORANGE JUICE CEASAR(1 LTR),LTR,41757,1-60/3,-30,-0.000043,Remove
9,1,SD37,SD37-FRUIT COCKTAIL JUICE (CEASER) 1LTR,PKT,35288,1-60/3,-57,-0.000081,Remove


Total cost impact on beverage flights economy equals SAR 1,040 and its impact on CPP equals SAR 0.0014869293193866986


##### BC beverage flight impact:

In [ ]:
# create beverages data frame for JC
bev_frame_bc = df_three[((df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(meal_exception))) | ((df_three['Block Time'] <= 89) & (df_three.ROUTE.isin(beverage_exception)) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
bev_frame_p_bc = df_p_three[((df_p_three['Block Time'] <= 89) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(meal_exception))) | ((df_p_three['Block Time'] <= 89) & (df_p_three.ROUTE.isin(beverage_exception)) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

# Remove duplicates rows
bev_frame_bc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)
bev_frame_p_bc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)

# drop unneeded columns
bev_frame_bc = bev_frame_bc.drop(columns=['CDN_NO','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
bev_frame_p_bc = bev_frame_p_bc.drop(columns=['CDN_NO','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

#Keep leg number 1 only 
bev_frame_bc = bev_frame_bc.drop(bev_frame_bc[bev_frame_bc.LEGNO.isin(['2', '3', '4'])].index)
bev_frame_p_bc = bev_frame_p_bc.drop(bev_frame_p_bc[bev_frame_p_bc.LEGNO.isin(['2', '3', '4'])].index)

#Change index values to SPEC_NO
bev_frame_bc.set_index(bev_frame_bc['SPECS_NO'], inplace = True)
bev_frame_p_bc.set_index(bev_frame_p_bc['SPECS_NO'], inplace = True)

# compare this month's items by last month's items, using the isin() function
for x, y in enumerate(bev_frame_bc.DBL_DATE.dt.day): 
    for i, j in enumerate(bev_frame_p_bc.DBL_DATE.dt.day):
        daliy_menu = bev_frame_bc[bev_frame_bc.DBL_DATE.dt.day == y]
        daliy_p_menu = bev_frame_p_bc[bev_frame_p_bc.DBL_DATE.dt.day == j]
        changes_list_bev_bc = pd.concat([changes_list_bev_bc, (daliy_menu[~(daliy_menu.isin(daliy_p_menu))] ), (daliy_p_menu[~(daliy_p_menu.isin(daliy_menu))])])

# remove empty rows using the dropna() function
changes_list_bev_bc.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)

#Create another frame for the new month to calculate the impact
bev_frame_bc_im = df_three[((df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(meal_exception))) | ((df_three['Block Time'] <= 89) & (df_three.ROUTE.isin(beverage_exception)) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
#Create another frame for the previous month to calculate the impact
bev_frame_p_bc_im = df_p_three[((df_p_three['Block Time'] <= 89) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(meal_exception))) | ((df_p_three['Block Time'] <= 89) & (df_p_three.ROUTE.isin(beverage_exception)) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

#Get the total cost of each new item
changes_list_bev_bc['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list_bev_bc['CPP_IMPACT'] = 0 # Add impact column to change_list
changes_list_bev_bc['STATUS'] = ''
bev_bc_cost_im = 0  # place holder for total cost impact for each item
for x,z in enumerate(changes_list_bev_bc['SPECS_NO']): # loop to get the specs number for new items
    if z in bev_frame_bc_im['SPECS_NO'].values:
        for i, y in enumerate(bev_frame_bc_im['PRICE'][bev_frame_bc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            bev_bc_cost_im = bev_bc_cost_im + y
            changes_list_bev_bc['STATUS'].iloc[x] = 'Add'
    else:
        for i, y in enumerate(bev_frame_p_bc_im['PRICE'][bev_frame_p_bc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            bev_bc_cost_im = bev_bc_cost_im - y
            changes_list_bev_bc['STATUS'].iloc[x] = 'Remove'
            
    # claculate the CPP for the current spec number    
    cpp_im = bev_bc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list_bev_bc['TOTAL_COST'].iloc[x] = bev_bc_cost_im
    # Add the CPP impact to the current spec number
    changes_list_bev_bc['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    bev_bc_cost_im = 0

    
# Remove duplicates rows
bev_frame_bc.drop_duplicates(subset=['SPECS_NO'], inplace= True)
bev_frame_p_bc.drop_duplicates(subset=['SPECS_NO'], inplace= True)

#Month variable
current_month = bev_frame_bc.DBL_DATE.dt.month_name().iloc[0]
previous_month = bev_frame_p_bc.DBL_DATE.dt.month_name().iloc[0]

# drop unneeded column
bev_frame_bc = bev_frame_bc.drop(columns=['DBL_DATE'])
bev_frame_p_bc = bev_frame_p_bc.drop(columns=['DBL_DATE'])
changes_list_bev_bc = changes_list_bev_bc.drop(columns=['DBL_DATE'])

#Reset index values 
bev_frame_bc.reset_index(drop= True, inplace = True)
bev_frame_p_bc.reset_index(drop= True, inplace = True)
changes_list_bev_bc.reset_index(drop= True, inplace = True)


sleep(1) # Time in seconds


if changes_list_bev_bc['ITEM_DESCRIPTION'].count() > 0:
    # print previous month dataframe
    print('{} menu:'.format(previous_month))
    display(bev_frame_p_bc)
    # print current month dataframe
    print('{} menu:'.format(current_month))
    display(bev_frame_bc)
    print('The following is the changes and impact breakdown:')
    display(changes_list_bev_bc)
    print('Total cost impact on meal flights business equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list_bev_bc['TOTAL_COST'].sum(), changes_list_bev_bc['CPP_IMPACT'].sum()))

else:
    # print previous month dataframe
    print('Previous month menu:')
    display(bev_frame_p_bc)
    # print current month dataframe
    print('Current month menu:')
    display(bev_frame_bc)
    print('No changes observed')

### Meal Flights:

#### YC meal flights impact:

In [105]:
# create beverages data frame for YC
meal_frame_yc = df_three[((df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(beverage_exception))) | ((df_three['Block Time'] > 89) & (df_three.ROUTE.isin(meal_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
meal_frame_p_yc = df_p_three[((df_p_three['Block Time'] > 89) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(beverage_exception))) | ((df_p_three['Block Time'] > 89) & (df_p_three.ROUTE.isin(meal_exception)) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

# Remove duplicates rows
meal_frame_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)
meal_frame_p_yc.drop_duplicates(subset=['SPECS_NO','DBL_DATE'], inplace= True)

# drop unneeded columns
meal_frame_yc = meal_frame_yc.drop(columns=['CDN_NO','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
meal_frame_p_yc = meal_frame_p_yc.drop(columns=['CDN_NO','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

#Keep leg number 1 only 
meal_frame_yc = meal_frame_yc.drop(meal_frame_yc[meal_frame_yc.LEGNO.isin(['2', '3', '4'])].index)
meal_frame_p_yc = meal_frame_p_yc.drop(meal_frame_p_yc[meal_frame_p_yc.LEGNO.isin(['2', '3', '4'])].index)

#Change index values to SPEC_NO
meal_frame_yc.set_index(meal_frame_yc['SPECS_NO'], inplace = True)
meal_frame_p_yc.set_index(meal_frame_p_yc['SPECS_NO'], inplace = True)


# compare this month's items by last month's items, using the isin() function
for x, y in enumerate(meal_frame_yc.DBL_DATE.dt.day): 
    for i, j in enumerate(meal_frame_p_yc.DBL_DATE.dt.day): 
        daliy_menu = meal_frame_yc[meal_frame_yc.DBL_DATE.dt.day == y]
        daliy_p_menu = meal_frame_p_yc[meal_frame_p_yc.DBL_DATE.dt.day == j]
        changes_list_meal_yc = pd.concat([(daliy_menu[~(daliy_menu.isin(daliy_p_menu))] ), (daliy_p_menu[~(daliy_p_menu.isin(daliy_menu))])])

    
# remove empty rows using the dropna() function
changes_list_meal_yc.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)

#Create another frame for the new month to calculate the impact
meal_frame_yc_im = df_three[((df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(beverage_exception))) | ((df_three['Block Time'] > 89) & (df_three.ROUTE.isin(meal_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
#Create another frame for the previous month to calculate the impact
meal_frame_p_yc_im = df_p_three[((df_p_three['Block Time'] > 89) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(beverage_exception))) | ((df_p_three['Block Time'] > 89) & (df_p_three.ROUTE.isin(meal_exception)) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

#Get the total cost of each new item
changes_list_meal_yc['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list_meal_yc['CPP_IMPACT'] = 0 # Add impact column to change_list
changes_list_meal_yc['STATUS'] = ''
meal_yc_cost_im = 0  # place holder for total cost impact for each item
for x,z in enumerate(changes_list_meal_yc['SPECS_NO']): # loop to get the specs number for new items
    if z in meal_frame_yc_im['SPECS_NO'].values:
        for i, y in enumerate(meal_frame_yc_im['PRICE'][meal_frame_yc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            meal_yc_cost_im = meal_yc_cost_im + y
            changes_list_meal_yc['STATUS'].iloc[x] = 'Add'
    else:
        for i, y in enumerate(meal_frame_p_yc_im['PRICE'][meal_frame_p_yc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            meal_yc_cost_im = meal_yc_cost_im - y
            changes_list_meal_yc['STATUS'].iloc[x] = 'Remove'
    # claculate the CPP for the current spec number    
    cpp_im = meal_yc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list_meal_yc['TOTAL_COST'].iloc[x] = meal_yc_cost_im
    # Add the CPP impact to the current spec number
    changes_list_meal_yc['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    meal_yc_cost_im = 0

    
# Remove duplicates rows
meal_frame_yc.drop_duplicates(subset=['SPECS_NO'], inplace= True)
meal_frame_p_yc.drop_duplicates(subset=['SPECS_NO'], inplace= True)

#Month variable
current_month = meal_frame_yc.DBL_DATE.dt.month_name().iloc[0]
previous_month = meal_frame_p_yc.DBL_DATE.dt.month_name().iloc[0]

# drop unneeded column
meal_frame_yc = meal_frame_yc.drop(columns=['DBL_DATE'])
meal_frame_p_yc = meal_frame_p_yc.drop(columns=['DBL_DATE'])
changes_list_meal_yc = changes_list_meal_yc.drop(columns=['DBL_DATE'])

#Reset index values 
meal_frame_yc.reset_index(drop= True, inplace = True)
meal_frame_p_yc.reset_index(drop= True, inplace = True)
changes_list_meal_yc.reset_index(drop= True, inplace = True)


sleep(1) # Time in seconds

if changes_list_meal_yc['ITEM_DESCRIPTION'].count() > 0:
    # print previous month dataframe
    print('{} menu:'.format(previous_month))
    display(meal_frame_p_yc)
    # print current month dataframe
    print('{} menu:'.format(current_month))
    display(meal_frame_yc)
    print('The following is the changes and impact breakdown:')
    display(changes_list_meal_yc)
    print('Total cost impact on meal flights economy equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list_meal_yc['TOTAL_COST'].sum(), changes_list_meal_yc['CPP_IMPACT'].sum()))

else:
    # print previous month dataframe
    print('Previous month menu:')
    display(meal_frame_p_yc)
    # print current month dataframe
    print('Current month menu:')
    display(meal_frame_yc)
    print('No changes observed')

<ipython-input-105-d452f7a48157>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meal_frame_yc.drop_duplicates(subset=['SPECS_NO'], inplace= True)
<ipython-input-105-d452f7a48157>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meal_frame_p_yc.drop_duplicates(subset=['SPECS_NO'], inplace= True)
C:\Users\kh-mo\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_b

January menu:


,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE
0,1,CMY2021,DISPOSABLE COVID-19 BOX SMALL (SV),PC,42893,100,0.00
1,1,SD1801,"TSC, SD1801-WATER CUP W/O LOGO HAYAT (125ML)",CUP,41289,100,0.41
2,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.38
3,1,SNYIC2040,S/W RECT CHICKEN SIXTY FIVE/THREE CHEESE-MAYONNAISE,PO,42854,100,8.16
4,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38
5,1,TL1609,MUFFIN DATES 90GR WRAPPED,PC,40620,100,3.31
6,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41
7,1,SNYIC2041,S/W RECT TURKEY SMOKED-PEPPERS/THREE CHEESE-,PO,42855,100,10.97
8,1,SNYIC2038,S/W CROISSANT THREE CHEESE-TOMATO SUNDRIED SALAD,PO,42856,100,9.21
9,1,CMY2025,SANITIZER BOTTLE 400ML (SV),BTL,43017,NaN,0.00


February menu:


,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE
0,1,CMY2021,DISPOSABLE COVID-19 BOX SMALL (SV),PC,42893,100,0.00
1,1,SD1801,"TSC, SD1801-WATER CUP W/O LOGO HAYAT (125ML)",CUP,41289,100,0.41
2,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.38
3,1,SNYIC2041,S/W RECT TURKEY SMOKED-PEPPERS/THREE CHEESE-,PO,42855,100,10.97
4,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38
5,1,TL1608,MUFFIN WHOLEMEAL-HONEY 90GR WRAPPED,PC,40619,100,3.31
6,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41
7,1,SNYIC2021,S/W RECT CHICKEN SHAWARMA/THREE CHEESE-MAYONNAISE WRAPPED,PO,42580,100,8.40
8,1,TL1611,MUFFIN VANILLA 90GR WRAPPED,PC,40844,100,3.06
9,1,SD1803,WATER MINERAL HAYAT (330ML),BTL,36798,NaN,0.54


The following is the changes and impact breakdown:


,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,TOTAL_COST,CPP_IMPACT,STATUS
0,1,SD1803,WATER MINERAL HAYAT (330ML),BTL,36798,NaN,0.54,43,0.000061,Add
1,1,SNYIH2102,S/W BREAD SUBMARINE PUMPKIN HALLOUMI-POTATO-TOMATO-OLIVES,PO,43052,49,8.38,179825,0.257103,Add
2,1,SNYIC2038,S/W CROISSANT THREE CHEESE-TOMATO SUNDRIED SALAD,PO,42856,100,9.21,-364187,-0.520693,Remove
3,1,#SDICE,"ICE, #SDICE-DRY ICE",KG,39897,100,7.56,-226,-0.000323,Remove
4,1,SD18R,SD18R-MANGO CEASER JUICE 1 LTR,PKT,41756,1-60/2,9.66,-38,-0.000054,Remove
5,1,SD37,SD37-FRUIT COCKTAIL JUICE (CEASER) 1LTR,PKT,35288,1-60/3,9.66,-57,-0.000081,Remove


Total cost impact on meal flights economy equals SAR -184,640 and its impact on CPP equals SAR -0.2639871437803462


#### BC meal flights impact:

In [ ]:
# create beverages data frame for JC
meal_frame_bc = df_three[((df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(beverage_exception))) | ((df_three['Block Time'] > 89) & (df_three.ROUTE.isin(meal_exception)) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
meal_frame_p_bc = df_p_three[((df_p_three['Block Time'] > 89) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(beverage_exception))) | ((df_p_three['Block Time'] > 89) & (df_p_three.ROUTE.isin(meal_exception)) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

# Remove duplicates rows
meal_frame_bc.drop_duplicates(subset=['ITEM_CODE','DBL_DATE'], inplace= True)
meal_frame_p_bc.drop_duplicates(subset=['ITEM_CODE','DBL_DATE'], inplace= True)

# drop unneeded columns
meal_frame_bc = meal_frame_bc.drop(columns=['CDN_NO','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
meal_frame_p_bc = meal_frame_p_bc.drop(columns=['CDN_NO','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

#Keep leg number 1 only 
meal_frame_bc = meal_frame_bc.drop(meal_frame_bc[meal_frame_bc.LEGNO.isin(['2', '3', '4'])].index)
meal_frame_p_bc = meal_frame_p_bc.drop(meal_frame_p_bc[meal_frame_p_bc.LEGNO.isin(['2', '3', '4'])].index)

#Change index values to SPEC_NO
meal_frame_bc.set_index(meal_frame_bc['SPECS_NO'], inplace = True)
meal_frame_p_bc.set_index(meal_frame_p_bc['SPECS_NO'], inplace = True)


# compare this month's items by last month's items, using the isin() function
for x, y in enumerate(meal_frame_bc.DBL_DATE.dt.day): 
    for x, y in enumerate(meal_frame_p_bc.DBL_DATE.dt.day): 
        daliy_menu = meal_frame_bc[meal_frame_bc.DBL_DATE.dt.day == y]
        daliy_p_menu = meal_frame_p_bc[meal_frame_p_bc.DBL_DATE.dt.day == j]
        changes_list_meal_bc = pd.concat([(daliy_menu[~(daliy_menu.isin(daliy_p_menu))] ), (daliy_p_menu[~(daliy_p_menu.isin(daliy_menu))])])

# remove empty rows using the dropna() function
changes_list_meal_bc.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)

#Create another frame for the new month to calculate the impact
meal_frame_bc_im = df_three[((df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(beverage_exception))) | ((df_three['Block Time'] > 89) & (df_three.ROUTE.isin(meal_exception)) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
#Create another frame for the previous month to calculate the impact
meal_frame_p_bc_im = df_p_three[((df_p_three['Block Time'] > 89) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(beverage_exception))) | ((df_p_three['Block Time'] > 89) & (df_p_three.ROUTE.isin(meal_exception)) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

#Get the total cost of each new item
changes_list_meal_bc['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list_meal_bc['CPP_IMPACT'] = 0 # Add impact column to change_list
changes_list_meal_bc['STATUS'] = ''
meal_bc_cost_im = 0  # place holder for total cost impact for each item
for x,z in enumerate(changes_list_meal_bc['SPECS_NO']): # loop to get the specs number for new items
    if z in meal_frame_bc_im['SPECS_NO'].values:
        for i, y in enumerate(meal_frame_bc_im['PRICE'][meal_frame_bc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            meal_bc_cost_im = meal_bc_cost_im + y
            changes_list_meal_bc['STATUS'].iloc[x] = 'Add'
    else:
        for i, y in enumerate(meal_frame_p_bc_im['PRICE'][meal_frame_p_bc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
            meal_bc_cost_im = meal_bc_cost_im - y
            changes_list_meal_bc['STATUS'].iloc[x] = 'Remove'
    # claculate the CPP for the current spec number    
    cpp_im = meal_bc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list_meal_bc['TOTAL_COST'].iloc[x] = meal_bc_cost_im
    # Add the CPP impact to the current spec number
    changes_list_meal_bc['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    meal_bc_cost_im = 0

    
# Remove duplicates rows
meal_frame_bc.drop_duplicates(subset=['ITEM_CODE'], inplace= True)
meal_frame_p_bc.drop_duplicates(subset=['ITEM_CODE'], inplace= True)

#Month variable
current_month = meal_frame_bc.DBL_DATE.dt.month_name().iloc[0]
previous_month = meal_frame_p_bc.DBL_DATE.dt.month_name().iloc[0]

# drop unneeded column
meal_frame_bc = meal_frame_bc.drop(columns=['DBL_DATE'])
meal_frame_p_bc = meal_frame_p_bc.drop(columns=['DBL_DATE'])
changes_list_meal_bc = changes_list_meal_bc.drop(columns=['DBL_DATE'])

#Reset index values 
meal_frame_bc.reset_index(drop= True, inplace = True)
meal_frame_p_bc.reset_index(drop= True, inplace = True)
changes_list_meal_bc.reset_index(drop= True, inplace = True)

sleep(1) # Time in seconds

if changes_list_meal_bc['ITEM_DESCRIPTION'].count() > 0:
    # print previous month dataframe
    print('{} menu:'.format(previous_month))
    display(meal_frame_p_bc)
    # print current month dataframe
    print('{} menu:'.format(current_month))
    display(meal_frame_bc)
    print('The following is the changes and impact breakdown:')
    display(changes_list_meal_bc)
    print('Total cost impact on meal flights economy equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list_meal_bc['TOTAL_COST'].sum(), changes_list_meal_bc['CPP_IMPACT'].sum()))

else:
    # print previous month dataframe
    print('Previous month menu:')
    display(meal_frame_p_bc)
    # print current month dataframe
    print('Current month menu:')
    display(meal_frame_bc)
    print('No changes observed')

## Summary

In [ ]:
total_cpm_impact = changes_list_bev_yc['CPP_IMPACT'].sum() + changes_list_bev_bc['CPP_IMPACT'].sum() + changes_list_meal_yc['CPP_IMPACT'].sum() + changes_list_meal_bc['CPP_IMPACT'].sum()
print(total_cpm_impact)

In [ ]:
actual_cpm_impact = avgCppNoHac - avgCppNoHacP
actual_cpm_impact